In [1]:
path = "c:/Users/Nathalie/Nextcloud/LADi/Orthografie Trainer/Code/04_bias_mitigation/"

import sys

sys.path.insert(1, path + "classes")

from DecisionTree_insession import in_session_decision_tree
from KNN_insession import in_session_KNN
from MLP_insession import in_session_MLP
from MetricsClass import Evaluation
from Plotting import Plots
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import pickle

# Ignore the warning message
warnings.filterwarnings("ignore", category=FutureWarning)

Learning Bias Mitigation: Parental Education Background <> DTE

In [2]:
# define feature cols
feature_cols = [
    "Erstloesung",
    "Schussel",
    "Erfolg",
    "Schwierigkeit",
    "ist_Schulzeit",
    "MehrfachFalsch",
    "vorher_abgebrochen",
    "Fehler",
    "Klassenstufe",
    "Jahredabei",
    "AnzahlAufgaben",
    "Sex__m",
    "Sex__w",
    "Testposition__pruefung",
    "Testposition__training",
    "Testposition__version",
    "Art__GK",
    "Art__GR",
    "Art__GZ",
    "Art__K",
    "Art__LB",
    "UserAttribut",
    "OrderNumber",
    "steps",
]

Decision Tree Classifier

In [3]:
DTE = in_session_decision_tree()
DTE.set_range(2, 61)
DTE.set_majority_group("AbiEltern")
DTE.set_demographic_category("AbiEltern")

DTE.load_matrices("06_learning_bias_mitigation/parental_education/matrix", ".pkl", True)

DTE.set_feature_cols(feature_cols)

DTE.set_majority_group("keinAbi")
DTE.set_minority_group("abi")

max_depth = [1, 5, 10, 15, 20, 25, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
min_samples_leaf = [1, 1, 1, 1, 1, 1, 1, 5, 10, 15, 20, 25, 1, 1, 1, 1, 1, 1]
min_samples_split = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 10, 15, 20, 25]
metrics = pd.DataFrame(
    columns=[
        "model",
        "group",
        "subgroup",
        "Length",
        "Sentence",
        "Accuracy",
        "Precision",
        "Recall",
        "AUC",
        "FPR",
        "max_depth",
        "min_samples_leaf",
        "min_samples_split",
    ]
)

for max_depth, min_samples_leaf, min_samples_split in zip(
    max_depth, min_samples_leaf, min_samples_split
):
    metrics_DTE = DTE.loop_matrices(
        "parental_education/matrix", max_depth, min_samples_leaf, min_samples_split
    )
    metrics_DTE["max_depth"] = max_depth
    metrics_DTE["min_samples_leaf"] = min_samples_leaf
    metrics_DTE["min_samples_split"] = min_samples_split

    metrics = pd.concat([metrics, metrics_DTE])

Created a Mitigation Object
Hi ML class


Evaluate

In [4]:
# evaluate metrics for fairness
evaluation_obj = Evaluation(metrics)
evaluation_obj.set_demographic_category("AbiEltern")
evaluation_obj.set_minority_group("abi")
evaluation_obj.set_majority_group("keinAbi")
index_list = ["max_depth", "min_samples_leaf", "min_samples_split", "Sentence"]
columns = ["max_depth", "min_samples_leaf", "min_samples_split"]

df_abi = evaluation_obj.evaluate_learning_bias(index_list, columns)

Created a Mitigation Object


In [9]:
met = ["EO", "PE", "PP", "SA"]
mean_table = pd.DataFrame()

index_ranges = [
    (2, 10, 8, "02-9"),
    (10, 20, 9, "10-19"),
    (20, 30, 9, "20-29"),
    (30, 40, 9, "30-39"),
    (40, 50, 9, "40-49"),
    (50, 60, 10, "50-60"),
]

for max_depth, min_samples_leaf, min_samples_split in zip(
    max_depth, min_samples_leaf, min_samples_split
):
    for x in met:
        for i_start, i_end, div, index_range in index_ranges:
            range_list = list(range(i_start, i_end))
            a = 0
            for i in range_list:
                a = a + df_abi[x][max_depth][min_samples_leaf][min_samples_split][i]
            a = a / div
            temp = pd.DataFrame(
                {
                    "Metrik": [x],
                    "Model": "DTE",
                    "Range": index_range,
                    "Val": a,
                    "max_depth": max_depth,
                    "min_samples_leaf": min_samples_leaf,
                    "min_samples_split": min_samples_split,
                }
            )
            mean_table = pd.concat([mean_table, temp])


## show results table
mean_table = pd.pivot_table(
    mean_table,
    values=["Val"],
    index=["max_depth", "min_samples_leaf", "min_samples_split", "Range"],
    columns=["Metrik", "Model"],
)

In [11]:
# show table
table = evaluation_obj.showTable(mean_table)
table